# Projet KAYAK

- on a une liste de villes en france
    - il faut récupérer la position GPS de la ville https://nominatim.org/ => dans un dataframe et sauver dans S3 en CSV
    - puis on récupère la météo 7j pour cette localisation https://openweathermap.org/api => sauver en CSV aussi
    - créer un critère pour la météo (selon la saison?) et les classer selon ce critère (daily.pop daily.rain temperature?)
    - ensuite trouver les 20 meilleurs hotels pour les top 10 destinations ayant la meilleure météo => scraping booking.com
    - stocker dans S3
    - afficher les meilleures destinations sur une carte avec plotly
    - petit moteur de recherche?
    



## 1) Phase Extract :
## 1.1) Récupération des coordonnées géographiques WGS84 des villes

In [2]:
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]


In [55]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()


True

In [51]:
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
DATA_DIR = "data"
DATA_DIR_CSV = f"{DATA_DIR}/csv"
DATA_DIR_HTML = f"{DATA_DIR}/html"
DATA_DIR_JSON = f"{DATA_DIR}/json"

In [4]:
base_url_nominatim = "https://nominatim.openstreetmap.org"

h = { 'User-Agent' : USER_AGENT}
p = { "format":"json"}

In [5]:
# grab town id + long/lat
def get_villes():
    for v in villes:
        p['q'] = v
        r = requests.get(url=f"{base_url_nominatim}/search", headers=h, params=p)
        json_ville = r.json()
        print(v)
        yield {
            'id': json_ville[0]['place_id'],
            'name': v,
            'lat': json_ville[0]['lat'],
            'lon': json_ville[0]['lon']
        }
cities = [ t for t in get_villes()]
df_cities = pd.DataFrame(cities)
df_cities.to_csv(f'{DATA_DIR_CSV}/cities.csv', encoding="utf-8")

Mont Saint Michel
St Malo
Bayeux
Le Havre
Rouen
Paris
Amiens
Lille
Strasbourg
Chateau du Haut Koenigsbourg
Colmar
Eguisheim
Besancon
Dijon
Annecy
Grenoble
Lyon
Gorges du Verdon
Bormes les Mimosas
Cassis
Marseille
Aix en Provence
Avignon
Uzes
Nimes
Aigues Mortes
Saintes Maries de la mer
Collioure
Carcassonne
Ariege
Toulouse
Montauban
Biarritz
Bayonne
La Rochelle


## 1.2) Récupération de la météo

In [1]:
# API DOC : https://openweathermap.org/forecast5
API_KEY_OWM = os.environ["API_KEY_OWM"]

NameError: name 'os' is not defined

In [ ]:

base_url_owm = "https://api.openweathermap.org/data/2.5"
lat=48.6359541
lon=-1.5114600
# il faut prendre au moins 6j x 8 valeurs/jour => ~50 jours
# en réalité l'API limite à 4j max
p = { 'lat': lat, 'lon': lon, 'cnt':100, 'units':'metric', 'appid': API_KEY_OWM }
r = requests.get(url=f"{base_url_owm}/forecast", headers=h, params=p)

# DEBUG
request_details = r.request
print(f"Request Method: {request_details.method}")
print(f"Request URL: {request_details.url}")
print(f"Request Headers: {request_details.headers}")

# 2) Phase Transform

## 2.1) récupérer et simplifier le JSON
- la meilleure stratégie c'est peut etre de mettre une journée de prévisions par fichier CSV sur le cloud
- ensuite on chargera tous les fichiers dans la base de données (boucle sur J+1 à J+4)

In [66]:
# récupérer le JSON le simplifier et le stocker sur le cloud sous forme de CSV

from datetime import datetime, timedelta
from glom import glom,T

tomorrow = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
tomorrow_tstamp = int(tomorrow.timestamp())

tomorrowPlus5d = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=5)
tomorrowPlus5d_tstamp = int(tomorrowPlus5d.timestamp())

print(f"weather from : {tomorrow} to {tomorrowPlus5d}")

def tstamp_to_date(tstamp):
    tstamp = datetime.fromtimestamp(tstamp)
    return tstamp.strftime("%Y-%m-%d")

def tstamp_to_time(tstamp):
    tstamp = datetime.fromtimestamp(tstamp)
    return tstamp.strftime("%H:%M:%S")

spec =  {
    "dt": "dt",
    "dt_str": "dt_txt",
    "date": ("dt", tstamp_to_date),
    "time": ("dt", tstamp_to_time),
    "temp": "main.temp",
    "feels_like": "main.feels_like",
    "temp_min": "main.temp_min",
    "temp_max": "main.temp_max",
    "pressure": "main.pressure",
    "humidity": "main.humidity",
    "weather_id": "weather.0.id",
    "weather_desc": "weather.0.main",
    "clouds" : "clouds.all",
    "wind_speed": "wind.speed",
    "rain_proba": "pop"
}        

wtf = None    
if r.status_code == 200 :

    with open(file=f'{DATA_DIR_JSON}/weather_forecast.json', mode='w', encoding='utf-8') as f:
        json.dump(obj=r.json(), fp=f, indent=4, ensure_ascii=True)

    #wtf = glom(target=r.json()['list'], spec=[spec])
    filtered =  [ elt for elt in r.json()['list'] if (elt['dt'] >= tomorrow_tstamp and elt['dt']<= tomorrowPlus5d_tstamp)]
    #wtf = glom(target=r.json()['list'], spec=T.filter(lambda x : x["dt"] >= tomorrow_tstamp).map(spec))
    wtf = glom(target=filtered, spec=[spec])
    print(json.dumps(obj=wtf, indent=2))
    # convert in Dataframe
    df_wtf = pd.DataFrame(wtf)
    #print(df_wtf)
    # export to CSV
    df_wtf.to_csv(path_or_buf=f'{DATA_DIR_CSV}/weather_forecasts.csv', header=True, sep=",", index=0, encoding="utf-8")
    # TODO upload to S3 with boto3
else:
    # TODO voir la gestion d'erreur de l'API
    print("Erreur lors de la récupération de la météo : {r.status_code}")
    print(r.text)


weather from : 2025-10-15 00:00:00 to 2025-10-19 00:00:00
[
  {
    "dt": 1760486400,
    "dt_str": "2025-10-15 00:00:00",
    "date": "2025-10-15",
    "time": "02:00:00",
    "temp": 10.71,
    "feels_like": 10.13,
    "temp_min": 10.71,
    "temp_max": 10.71,
    "humidity": 88,
    "weather_id": 800,
    "weather_desc": "Clear",
    "clouds": 7,
    "wind_speed": 3.79,
    "rain_proba": 0
  },
  {
    "dt": 1760497200,
    "dt_str": "2025-10-15 03:00:00",
    "date": "2025-10-15",
    "time": "05:00:00",
    "temp": 10.52,
    "feels_like": 9.87,
    "temp_min": 10.52,
    "temp_max": 10.52,
    "humidity": 86,
    "weather_id": 800,
    "weather_desc": "Clear",
    "clouds": 0,
    "wind_speed": 4.11,
    "rain_proba": 0
  },
  {
    "dt": 1760508000,
    "dt_str": "2025-10-15 06:00:00",
    "date": "2025-10-15",
    "time": "08:00:00",
    "temp": 9.47,
    "feels_like": 7.39,
    "temp_min": 9.47,
    "temp_max": 9.47,
    "humidity": 90,
    "weather_id": 800,
    "weather_desc

# Définition d'un scoring de météo
Ces seuils et scores sont issus d’études climatologiques (ICT de Mieczkowski) et de standards utilisés par les institutions météorologiques, adaptés aux besoins du tourisme pour offrir un indice synthétique fiable

## 1) classification par tranche

### a) Température
- <0 : très froid, score 0/10
- 0-10 : froid, score 2/10
- 11-17 : frais, score 5/10
- 18-28 : idéal, score 10/10
- 29-33 : chaud, score 5/10
- \>33 : très chaud, score 0/10

### b) Précipitations: proba uniquement (pas de mm de précipitation)
- 0 mm : Sec, idéal, score 10/10.
- 0,1-1 mm : Faible pluie/bruine, score 8/10.
- 1-10 mm : Pluie modérée, score 5/10.
- 11-30 mm : Pluie forte, score 2/10.
- 30 mm : Pluie très forte, score 0/10.

Mais le niveau de précipitation n'est pas présent dans les données gratuires, donc on ne gardera que la probabilité en % de précipitation.
On prendra la valeur que l'on ramènera à une note /10.

### c) Vent
- 0-15 km/h : Agréable ou léger, score 10/10.
- 16-28 km/h : Modéré, score 8/10.
- 29-38 km/h : Bonne brise, score 5/10.
- 39-61 km/h : Vent frais ou grand frais, score 2/10.
- 61 km/h : Fort coup de vent/tempête, score 0/10.

### d) couverture nuageuse
- 0-4 (dégagé à généralement dégagé) : score 10/10.
- 5-9 (généralement nuageux) : score 6/10.
- 10 (nuageux) : score 2/10

### e) humidité relative
- 40-70% : Confortable, score 10/10.
- 70-80% : Humide, score 7/10.
- 20-39% : Sec, score 7/10.
- \<20% ou >80% : Très inconfortable, score 3/10

### f) pression atmosphérique

Pression atmosphérique (hPa)  |  Score /10  |  Interprétation                                 
| :-- | :-- | :-- |
< 980                         |  1          |  Dépression sévère : mauvais temps probable     
980–995                       |  3          |  Basse pression : instabilité, risque pluie/vent
996–1005                      |  6          |  Légère instabilité à modérée                   
1006–1015                     |  10         |  Zone idéale, temps stable                      
1016–1025                     |  8          |  Haute pression, possible ciel dégagé           
1026–1035                     |  6          |  Anticyclone marqué, air sec, risque canicule   
\>1035                         |  3          |  Très forte pression : canicule ou brouillard   


### g) description générale du temps

difficilement utilisable et difficile à synthétiser/agréger...

- 2xx : thunderstorm
- 3xx : drizzle
- 5xx : rain
- 6xx : snow
- 7xx : atmosphere
- 8xx : clear

## 2) Pondération et aggrégations des résultats par prévision

### a) Définir des poids pour chaque critère selon la destination et la saison (par exemple, la pluie vaut plus pour la visite de monuments).

| Critère | Pondération (%) | Justification |
| :-- | :-- | :-- |
| Température | 30 | Crucial pour le confort et la sécurité lors des activités extérieures |
| Précipitations | 30 | Forte pluie, neige ou grêle impactent négativement la plupart des activités touristiques. |
| Vent | 15 | Vent fort gêne les déplacements, crée des risques pour certaines activités. |
| Couverture nuageuse | 10 | Les journées très couvertes ou brumeuses affectent la visibilité et l’ambiance. |
| Humidité | 10 | Taux d’humidité élevé ou très faible peut altérer le confort. |
| Pression atmosphérique | 5 | Indice complémentaire, associé aux conditions (anticyclone/dépression) |

### b) Calculer un score global pour chaque journée
on peut soit
- pour chaque tranche horaire (3h), puis agréger pour chaque journée (max, moyenne, pondérée selon l’importance de la journée au programme)
- retenir uniquement la température à 12h ce qui permet ne garder qu'une seule valeur

### c) Proposer un score final pour la période de 4 jours

Score final = (Température×0.3) + (Précipitations×0.3) + (Couverture nuageuse×0.15) + (Vent×0.15) + (Humidité×0.1) + (Pression atmox 0.05)

Ensuite, Par exemple, moyenne des scores journaliers, ou la note minimale pour éviter qu’une mauvaise journée n’annule tout séjour.

Proposition d'une échelle de 0 à 100, où :
- 80-100 : Conditions idéales pour le tourisme
- 60-79  : Conditions acceptables, quelques désagréments possibles
- 40-59  : Conditions médiocres, tourisme possible mais limité
- 0-39   : Conditions défavorables, tourisme déconseillé

# 3) Scraping Booking.com

TODO
ne pas oublier de mettre
- User-Agent
- Accept-Language
- Accept-Encoding
- Content-Length


In [3]:
from scrapy import Selector

In [6]:
h = {
    'User-Agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding' : 'gzip, deflate, br'
}

In [ ]:
url_booking = "https://www.booking.com/searchresults.fr.html"
#url_booking = "https://www.booking.com/index.fr.html"
#p = { 'ss':'Amiens', 'ht_id=204': 'ht_id=204'}
p = { 'ss': 'Brest'}

# TODO mettre un try except avec un r.raise_error()
r = requests.get(url=url_booking, params=p, headers=h)
if r.status_code != 200:
    print(f"error {r.status_code}")

display(r.text)

In [7]:
# écriture des fichiers bruts HTML pour s'assurer qu'on a des data meme si blacklisté
with open(file=f"data/booking_raw_{p['ss']}.html", mode='w', encoding='utf-8') as f:
    f.write(r.text)


In [11]:
selector = Selector(text=r.text, type="html", encoding="utf-8")

hotel_a_class = "bd77474a8e"
selector_hotel_link = selector.css(f"a[class={hotel_a_class}]::attr(href)").extract()
hotel_links = [ link.split("?")[0] for link in selector_hotel_link[:20] ]

for link in hotel_links:
    print(link)

NameError: name 'r' is not defined

In [12]:
# dans la page de l'hotel :
# on teste sur un hotel
#url_hotel = hotel_links[0]
url_hotel = 'https://www.booking.com/hotel/fr/le-barracuda-amp-spa.fr.html'
print(type(url_hotel))

selector = None
try:
    r_hotel = requests.get(url=url_hotel, headers=h)
    r_hotel.raise_error()
    display(r_hotel.text)
    selector = Selector(text=r_hotel.text, type="html", encoding="utf-8")
    with open(file=f"data/booking_raw_{p['ss']}_hotel_01.html", mode='w', encoding='utf-8') as f:
        f.write(r_hotel.text)
except:
    print(f"error {r_hotel.status_code}")

<class 'str'>
error 202


In [50]:
with open('data/booking_raw_Brest_hotel.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

# Créer un objet Selector
selector = Selector(text=html_content, type="html", encoding="utf-8")

# le nom de l'hotel
# <h2 class="ddb12f4f86 pp-header__title">Le Barracuda &amp; Spa, Centre Port, pieds dans l'eau, vue mer</h2>
hotel_name_class = "ddb12f4f86 pp-header__title"
hotel_name = selector.css(f"h2[class='{hotel_name_class}']::text").get()
print(hotel_name)

# score / note/10
# .c465553d73 > div:nth-child(2)
#  <div aria-hidden="true" class="f63b14ab7a dff2e52086">8,6</div>
hotel_score_class = "f63b14ab7a dff2e52086"
hotel_score = selector.css(f"div[class='{hotel_score_class}']::text").extract()[0]
print(hotel_score)

# text de description de l'hotel
#<p data-testid="property-description" class="b99b6ef58f f1152bae71">
#L’établissement Le Barracuda &amp; Spa, Centre Port, pieds dans l'eau, vue mer vous accueille à Brest, à 500 mètres de ce lieu d’intérêt : Musée national de la Marine, Brest. Il comprend un restaurant, ainsi qu’un spa et centre de bien-être avec une piscine intérieure, une salle de sport et un sauna. Une connexion Wi-Fi gratuite est fournie dans tout l’établissement et un parking privé est disponible sur place.
#Les hébergements comprennent la climatisation et une télévision à écran plat. Vous bénéficierez d’une cuisine entièrement équipée avec un coin repas, ainsi que d’une salle de bains privative comportant une douche, des articles de toilette gratuits et un sèche-cheveux. Un réfrigérateur, un lave-vaisselle et un micro-ondes sont également à votre disposition, de même qu’une machine à café et une bouilloire.
#L’établissement Le Barracuda &amp; Spa, Centre Port, pieds dans l'eau, vue mer sert un petit-déjeuner buffet ou continental.
#Cet établissement 4 étoiles dispose d’un hammam et d’une terrasse.
#Vous séjournerez à proximité de ces lieux d’intérêt&nbsp;: Château de Brest, Gare de Brest et Arsenal de Brest. L'aéroport le plus proche (Aéroport Brest Bretagne) est à 10 km.</p>
hotel_desc_class = "b99b6ef58f f1152bae71"
hotel_desc = selector.css(f"p[class='{hotel_desc_class}']::text").extract()[0]
print(hotel_desc)
print(len(hotel_desc))

# la localisation lat,long
#   <a id=map_trigger_header_pin 
#   <a data-atlas-latlng="48.380838,-4.488184"
# data-atlas-bbox="48.36286852595623,-4.515224962877822,48.39880113290929,-4.461123942390796" 
# data-source="top_link"
#  title="Le Barracuda &amp; Spa, Centre Port, pieds dans l'eau, vue mer, Brest - Vérifier l'emplacement" id="map_trigger_header_pin" href="#map_opened-map_trigger_header_pin" class="de576f5064 bef8628e61 a4b276b5a9 e0d8514d4d"><span><span class="fc70cba028 e2a1cd6bfe" aria-hidden="true"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" width="50px"><path d="M12 0a8.01 8.01 0 0 0-8 8c0 3.51 5 12.025 7.148 15.524A1 1 0 0 0 12 24a.99.99 0 0 0 .852-.477C15 20.026 20 11.514 20 8a8.01 8.01 0 0 0-8-8m0 11.5A3.5 3.5 0 1 1 15.5 8a3.5 3.5 0 0 1-3.5 3.5"></path></svg></span></span></a>
hotel_loc = selector.css("#map_trigger_header_pin").css("a[data-atlas-latlng]::attr(data-atlas-latlng)").get()
print(hotel_loc)
(hotel_lat, hotel_lon) = hotel_loc.split(",")
print(f"Long={hotel_lon} Lat={hotel_lat}")

Le Barracuda & Spa, Centre Port, pieds dans l'eau, vue mer
8,6
L’établissement Le Barracuda & Spa, Centre Port, pieds dans l'eau, vue mer vous accueille à Brest, à 500 mètres de ce lieu d’intérêt : Musée national de la Marine, Brest. Il comprend un restaurant, ainsi qu’un spa et centre de bien-être avec une piscine intérieure, une salle de sport et un sauna. Une connexion Wi-Fi gratuite est fournie dans tout l’établissement et un parking privé est disponible sur place.

Les hébergements comprennent la climatisation et une télévision à écran plat. Vous bénéficierez d’une cuisine entièrement équipée avec un coin repas, ainsi que d’une salle de bains privative comportant une douche, des articles de toilette gratuits et un sèche-cheveux. Un réfrigérateur, un lave-vaisselle et un micro-ondes sont également à votre disposition, de même qu’une machine à café et une bouilloire.

L’établissement Le Barracuda & Spa, Centre Port, pieds dans l'eau, vue mer sert un petit-déjeuner buffet ou continen